In [1]:
# ! pip install mlxtend
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

sales_fact_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_1998.csv")
sales_fact_dec_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_dec_1998.csv")
product_ori_df = pd.read_csv("./hw1_data/product.csv")
# display(sales_fact_1998_ori_df.info(), sales_fact_dec_1998_ori_df.info(), product_ori_df.info())

time: 407 ms (started: 2022-01-04 14:30:52 +00:00)


In [2]:
transaction_ori_df = pd.concat([sales_fact_1998_ori_df, sales_fact_dec_1998_ori_df]).reset_index()
transaction_df = transaction_ori_df.copy().drop(['promotion_id', 'unit_sales', 'index', 'store_sales', 'store_cost'], axis=1)
transaction_df['product_id'] = transaction_ori_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0])
transaction_df = pd.concat([transaction_df, pd.get_dummies(transaction_df['product_id'])], axis=1).drop(['product_id'],axis=1)
transaction_df = transaction_df.drop_duplicates()
transaction_df = transaction_df.groupby(['time_id', 'customer_id', 'store_id']).sum()

time: 33.1 s (started: 2022-01-04 14:30:52 +00:00)


In [3]:
# an_col = [col for col in transaction_df.columns if (set(transaction_df[col].unique().tolist()) != {0,1})]
# duplicate_ind = []
# for ind in transaction_df.loc[(transaction_df==2).any(axis=1),an_col].index:
#     cond = np.logical_and(np.logical_and(transaction_ori_df['time_id']==ind[0], transaction_ori_df['customer_id']==ind[1]), transaction_ori_df['store_id']==ind[2])
#     tmp = transaction_ori_df.loc[cond , 'product_id'].duplicated()
#     duplicate_ind.append(tmp.loc[tmp==1].index.values[0])
    
# duplicate_ind

[]

time: 241 ms (started: 2022-01-04 14:31:25 +00:00)


In [9]:
apriori_freq_itemset = apriori(transaction_df, min_support=0.0001, use_colnames=True, low_memory=True)
apriori_rules = association_rules(freq_itemset, metric='confidence', min_threshold=0.9)
display(apriori_freq_itemset.sort_values(by = 'support'))
display(apriori_rules.sort_values(by = 'lift', ascending=False).head(15))
apriori_rules.sort_values(by = 'lift', ascending=False).head(15).to_csv('apriori_product.csv')

,support,itemsets
16070,0.000106,"(Faux Products HCL Nasal Spray, Just Right Can..."
7112,0.000106,"(Fort West Golden Raisins, Colossal Manicotti)"
7113,0.000106,"(Giant Large Brown Eggs, Colossal Manicotti)"
7114,0.000106,"(Golden Lime Popsicles, Colossal Manicotti)"
12403,0.000106,"(High Top Red Pepper, Tri-State Firm Tofu)"
...,...,...
311,0.003936,(Carrington Ice Cream)
748,0.003963,(Great Pumpernickel Bread)
1389,0.003989,(Super Chunky Peanut Butter)
746,0.004095,(Great English Muffins)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1722,"(Red Spade Low Fat Bologna, Better Large Canne...","(Tell Tale Mushrooms, PigTail Frozen Sausage P...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
2028,"(Hilltop Conditioning Shampoo, Cormorant AA-Si...","(Faux Products Extra Moisture Shampoo, Horatio...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
2040,"(Horatio Fudge Cookies, Tri-State Tangerines)","(Faux Products Extra Moisture Shampoo, Cormora...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1742,"(Horatio Fudge Cookies, Cormorant AA-Size Batt...","(Faux Products Extra Moisture Shampoo, Tri-Sta...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1304,"(Horatio Fudge Cookies, Cormorant AA-Size Batt...","(Hilltop Conditioning Shampoo, Tri-State Tange...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1740,"(Faux Products Extra Moisture Shampoo, Cormora...","(Horatio Fudge Cookies, Tri-State Tangerines)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1739,"(Faux Products Extra Moisture Shampoo, Cormora...","(Hilltop Conditioning Shampoo, Tri-State Tange...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1058,"(Better Canned Tuna in Water, Ebony Potatos)","(Urban Large Eggs, Just Right Chicken Soup)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1303,"(Hilltop Conditioning Shampoo, Tri-State Tange...","(Horatio Fudge Cookies, Cormorant AA-Size Batt...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1060,"(Urban Large Eggs, Just Right Chicken Soup)","(Better Canned Tuna in Water, Ebony Potatos)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf


time: 3.46 s (started: 2022-01-04 14:49:33 +00:00)
